<a href="https://colab.research.google.com/github/JonathanVillarrubia/data-engineer-coderhouse/blob/main/Entregable_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importación de librerias

import requests
import psycopg2

# Configuración de la API y la consulta: Me traigo los datos especificos para la provincia de Buenos Aires
url = "https://weatherapi-com.p.rapidapi.com/current.json"
querystring = {"q": "Buenos Aires"}
headers = {
    "X-RapidAPI-Key": "da9f357533msh3654b433b52c327p19ff8djsne4d1d5ce47b0",
    "X-RapidAPI-Host": "weatherapi-com.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)
data = response.json()

# Conexión a Amazon Redshift
redshift_host = "data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com"
redshift_port = "5439"
redshift_database = "data-engineer-database"
redshift_user = "jonathan_villarrubia_coderhouse"
redshift_password = "PVaF53JIB1"

conn = psycopg2.connect(
    host=redshift_host,
    port=redshift_port,
    dbname=redshift_database,
    user=redshift_user,
    password=redshift_password
)

# Creo un cursor
cur = conn.cursor()

# Creo la tabla en la base de datos
create_table_query = """
    CREATE TABLE IF NOT EXISTS weather_data_bs_as (
        location_name VARCHAR(255),
        region VARCHAR(255),
        country VARCHAR(255),
        latitude DECIMAL,
        longitude DECIMAL,
        timezone_id VARCHAR(255),
        localtime_epoch BIGINT,
        localtime_text VARCHAR(255),  -- Cambio acá: localtime -> localtime_text
        last_updated_epoch BIGINT,
        last_updated VARCHAR(255),
        temp_c DECIMAL,
        temp_f DECIMAL,
        is_day INT,
        condition_text VARCHAR(255),
        wind_mph DECIMAL,
        wind_kph DECIMAL,
        wind_degree INT,
        wind_dir VARCHAR(255),
        pressure_mb DECIMAL,
        pressure_in DECIMAL,
        precip_mm DECIMAL,
        precip_in DECIMAL,
        humidity INT,
        cloud INT,
        feelslike_c DECIMAL,
        feelslike_f DECIMAL,
        vis_km DECIMAL,
        vis_miles DECIMAL,
        uv DECIMAL,
        gust_mph DECIMAL,
        gust_kph DECIMAL
    );
"""
cur.execute(create_table_query)
conn.commit()

# Inserto datos en la tabla creada
insert_query = """
    INSERT INTO weather_data_bs_as (
        location_name, region, country, latitude, longitude, timezone_id,
        localtime_epoch, localtime_text, last_updated_epoch, last_updated,
        temp_c, temp_f, is_day, condition_text, wind_mph, wind_kph,
        wind_degree, wind_dir, pressure_mb, pressure_in, precip_mm, precip_in,
        humidity, cloud, feelslike_c, feelslike_f, vis_km, vis_miles, uv,
        gust_mph, gust_kph
    )
    VALUES (
        %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
        %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
    );
"""

data_to_insert = (
    data["location"]["name"], data["location"]["region"], data["location"]["country"],
    data["location"]["lat"], data["location"]["lon"], data["location"]["tz_id"],
    data["location"]["localtime_epoch"], data["location"]["localtime"],
    data["current"]["last_updated_epoch"], data["current"]["last_updated"],
    data["current"]["temp_c"], data["current"]["temp_f"], data["current"]["is_day"],
    data["current"]["condition"]["text"], data["current"]["wind_mph"], data["current"]["wind_kph"],
    data["current"]["wind_degree"], data["current"]["wind_dir"], data["current"]["pressure_mb"],
    data["current"]["pressure_in"], data["current"]["precip_mm"], data["current"]["precip_in"],
    data["current"]["humidity"], data["current"]["cloud"], data["current"]["feelslike_c"],
    data["current"]["feelslike_f"], data["current"]["vis_km"], data["current"]["vis_miles"],
    data["current"]["uv"], data["current"]["gust_mph"], data["current"]["gust_kph"]
)
cur.execute(insert_query, data_to_insert)
conn.commit()

# Cierro la conexión a Redshift
cur.close()
conn.close()

print("Proceso completado: Datos del clima insertados en la tabla de Redshift.")


Proceso completado: Datos del clima insertados en la tabla de Redshift.
